In [1]:
import pandas as pd
import numpy as np
import abydos
from abydos import distance
import re
import math

# General Instruction

- Cleanup dataset based on the information that is given:
You need to clean the dataset according to the information that is given to you. This means that there are problems with the dataset that need to be fixed, and you should use the information given to you to determine what those problems are and how to fix them.

- Each case has different data quality problems, there will be hint and additional information that can help you understand the problem:
Each row in the dataset may have different data quality problems. There will be hints and additional information provided to help you understand what the specific problem is with each row.

- You can do any approach on cleaning the data, but you should clean the instructed column only:
You have the freedom to use any approach to clean the data, but you should only clean the instructed column. This means that you should not modify any other columns in the dataset, or add or remove any rows.

- Do not create new column or remove any column. Also do not create new row, or remove any row:-
You are not allowed to create new columns or remove any columns from the dataset. You are also not allowed to add or remove any rows.

- Each column will have a flag column something equivalent to <column\_name>\_flag. This column can be used to flag the row if you want to not include it to the downstream task. 0: safe_flag, 1: delete_flag, 2: null_flag (if you want to still include the row with null treatment). You can also add a new category but please add justification and explanation of the new category, there are three categories you can use:
safe_flag (0): this row is safe to use in downstream tasks
delete_flag (1): this row should be deleted and not used in downstream tasks
null_flag (2): this row can be included in downstream tasks but with null treatment.
You can also add a new category, but you need to provide a justification and an explanation for the new category. It is worth to note that the completeness of the dataset is also matter, so try not to flag to many things, and do your best to clean the values.

- For each data cleaning task, we have provided a function that represents the goal of the cleaning. For example, clean_duplicate_id(df) is the function for removing duplicate ID values. These functions take a DataFrame as input and return the cleaned version of the DataFrame.

    In each chunk of data cleaning task, you will see the following three parts:

    1. The clean_<name> function that performs the specific cleaning task.
    2. The execution of the cleaning function on the DataFrame.
    3. A checking part to help you evaluate the effectiveness of the cleaning.
    
  While you can create new cells and add additional code, the cleaning must be performed through the provided cleaning functions. You can adjust the order of the cleaning steps, but please try to move the whole chunks of code to avoid any errors.

The cleaning task will be considered complete if this notebook can be run sequentially by executing "restart and runall"




# Purpose
The purpose of this dataset is to conduct exploratory analysis of the listings and create a prediction model for listing price using some columns from the dataset. This means that the dataset is intended to be used to explore the characteristics and features of the listings, and to build a model that can predict the price of a listing based on certain variables in the dataset. The goal is to gain insights into the factors that influence the price of a listing and to develop a model that can accurately predict listing prices based on those factors.

# Columns and Dataset Description
- id: a unique identifier for each listing.
- name: the name or title of the listing, as provided by the host.
- host_id: a unique identifier for each host.
- host_name: the name of the host who listed the property.
- neighbourhood_group: the larger geographic area in which the listing is located (e.g. a borough or group of neighborhoods).
- neighbourhood: the specific neighborhood in which the listing is located.
- latitude: the latitude coordinate of the listing.
- longitude: the longitude coordinate of the listing.
- room_type: the type of space that is being listed (e.g. an entire apartment, a private room, a shared room).
- price: the nightly price of the listing, in the currency specified in the dataset.
- minimum_nights: the minimum number of nights that a guest must book the listing for.
- number_of_reviews: the total number of reviews that the listing has received.
- last_review: the date of the most recent review of the listing.
- reviews_per_month: the average number of reviews per month that the listing has received.
- calculated_host_listings_count: the total number of listings that the host has on Airbnb.
- availability_365: the number of days per year that the listing is available for booking.
- number_of_reviews_ltm: the total number of reviews that the listing has received in the last 12 months.
- license: a license number for the listing, if applicable (this column may not be present in all versions of the dataset).

Besides the columns above, there are columns pre-defined for flagging the rows based on particular data cleaning context:
- id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the id column (duplicate).
- host_id_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the host_id column.
- neighbourhood_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the neighbourhood column.
- latitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the latitude column.
- longitude_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the longitude column.
- minimum_nights_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the minimum_nights column.
- number_of_reviews_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the number_of_reviews column.
- last_review_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the last_review column.
- room_type_flag: a flag column indicating whether a given row should be included in downstream analysis or not based on data quality issues related to the room_type column.

# Load Data

In [2]:
airbnb_pd = pd.read_csv("chicago_vert_2_a.csv")

In [3]:
airbnb_pd.columns

Index(['Unnamed: 0', 'id', 'listing_url', 'scrape_id', 'last_scraped',
       'source', 'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights

In [4]:
airbnb_pd.columns

Index(['Unnamed: 0', 'id', 'listing_url', 'scrape_id', 'last_scraped',
       'source', 'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights

In [5]:
airbnb_pd.neighbourhood.unique()

array(['Chicago, Illinois, United States',
       'Chicago, Illinois, UnitejduV States', nan,
       'Chicago, vIlljinois, Uniated States',
       'Chicagoy,MK Illinois, United States',
       'Chicago, IllinoisTc, United SRtates',
       'ChiNciacgo, Illinois, United States',
       'rChicago, Illinosis, United SEtates',
       'Chicago, GIllinoisK, United Strates',
       'Chicago, Illinois,N fOUnited States',
       'hChicago, Illinoiss, Unilted States',
       'aChicago, Illinois, qUnited Staktes',
       'ChVicago, Illinois, UnitqedV States',
       'Chicago, Illinois, Unipted eStatPes',
       'Chicago, Illinois, Ubnited MStaetes', 'lbndan',
       'rChicago, Illinois, UnitedW Satates',
       'NCQhicago, Illinois, UniteMd States',
       'ChiPcago, Illinois,y UnTited States',
       'Chicago, Illinois, United  Strlates', 'ncaNwn',
       'CShicagGo, Illinois, Unbited States',
       'Chicagos, Illinoxis,U United States',
       'Chicago, ImlulYinois, United States',
       'Chbi

# cleanup duplicate id
The ID column must contain unique values. If there are any duplicate values in this column, you will need to take action to ensure that each ID is unique. You can do this by either fixing the duplicates (if you want to keep them) or by flagging them for removal (1) using the id_flag column.

In [6]:
def clean_duplicate_id(df):
    dup_ids = df[df.id_flag==0]
    dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
    dup_ids = dup_ids[dup_ids.name>1]
    
    for id_ in list(dup_ids["id"]):
        temp_df=df[df["id"]==id_]
        
        try:
            id_=list(temp_df[temp_df.duplicated()].index)[0]  ## two rows are completely the same
            df.loc[id_,"id_flag"]=1
        except:
            index_ls=list(temp_df.index)   ## one row combine with another row equals to one of the original row
            temp_row_1=temp_df.loc[index_ls[0]]
            temp_row_2=temp_df.loc[index_ls[1]]
            row_comb=temp_df.loc[index_ls[0]].combine_first(temp_df.loc[index_ls[1]])
            try:
                comp_result_1=list(set(list(temp_row_1.fillna(0)==row_comb.fillna(0))))[0]
                comp_result_2=list(set(list(temp_row_2.fillna(0)==row_comb.fillna(0))))[0]
                if comp_result_1==True:
                    df.loc[index_ls[1],"id_flag"]=1
                elif comp_result_2==True:
                    df.loc[index_ls[0],"id_flag"]=1
                else:
                    df.loc[index_ls[0],"id_flag"]=2
                    df.loc[index_ls[1],"id_flag"]=2  ##mark types which not belongs to the types mentioned above
            except:
                print(id_)
    return df

In [7]:
airbnb_pd = clean_duplicate_id(airbnb_pd)

# Duplicate IDS checking 
To ensure that all ID values in the dataset are unique, you should check for duplicate IDs. When you run the query to check for duplicates, there should be no rows returned, indicating that there are no duplicate ID values present in the dataset.

In [8]:
dup_ids = airbnb_pd[airbnb_pd.id_flag==0]
dup_ids = dup_ids.groupby("id").count()[["name"]].reset_index()
dup_ids = dup_ids[dup_ids.name>1]
dup_ids

,id,name


# cleanup inconsistent host id
Each host_id value in the dataset should be associated with only one host_name. However, there may be inconsistencies in the dataset where a host_id is associated with different host_name values.

To clean this up, you can either change the host_name value to a consistent value based on information in the dataset, or flag the host_id_flag column to indicate that the row should be removed from downstream tasks.

For example, if you find that a host_id is associated with multiple host_name values, you may want to investigate further to determine which host_name is correct. If one of the host_name values is clearly incorrect (e.g., a misspelling or a name that does not match the owner of the property), you could update the host_name value to the correct value.

Alternatively, if you cannot determine the correct host_name value, or if you want to exclude the row from downstream tasks for other reasons, you can flag the host_id_flag column with a value of 1 to indicate that the row should be removed.

In [9]:
def clean_host_id(df):
    dup_host_id = df[df.host_id_flag==0]
    dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
    dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
    dup_host_id_df=dup_host_id[dup_host_id["id"]>1]
    dup_host_id_ls=list(dup_host_id_df["host_id"])
    for dup_host_id in dup_host_id_ls:
        temp_df=df[df["host_id"]==dup_host_id]
        temp_id_ls=list(temp_df.index)
        
        temp_match_ls=[]
        for id_ in temp_id_ls:
            string=temp_df.loc[id_,"host_name"]
            pattern = r"^[A-Z][a-z0-9_-]{3,19}$"  # here is only a simple way to clean the host_id
            match=re.match(pattern, string)
            if match:
                temp_match_ls.append(string)
            else:
                df.loc[id_,"host_id_flag"]=1  # while this method is also not perfect, since under same host_id, the host_name could be all wrong
        
        if len(set(temp_match_ls))!=1:
            for id_ in temp_id_ls:
                 df.loc[id_,"host_id_flag"]=2
    return df

In [10]:
airbnb_pd = clean_host_id(airbnb_pd)

# Inconsistent Host ID checking 

This query should return zero rows once you implement the cleaning process

In [11]:
dup_host_id = airbnb_pd[airbnb_pd.host_id_flag==0]
dup_host_id = dup_host_id.groupby(["host_id","host_name"]).count()[["id"]].reset_index()
dup_host_id = dup_host_id.groupby("host_id").count()["id"].reset_index()
dup_host_id[dup_host_id["id"]>1]

,host_id,id


In [12]:
## I have identified other issues, 1) the two names are pretty close to each other (which is very hard to detect 2) we can choose the right name based on the frequency

# cleanup neighbourhood
The neighbourhood column in the dataset should contain values that match the neighbourhoods defined in the official neighbourhood_list. However, there may be some values in the neighbourhood column that are incorrect due to errors or noise in the data.

To clean up the neighbourhood column, you can try to match each value in the column to a valid neighbourhood in the neighbourhood_list using a string distance function such as abydos. If you can successfully match a value in the neighbourhood column to a neighbourhood in the neighbourhood_list, you can replace the value in the dataset with the correct neighbourhood name.

However, if you are unsure about how to clean up a particular value in the neighbourhood column, or if you cannot match the value to a valid neighbourhood in the neighbourhood_list, you can flag the row for deletion by setting the neighbourhood_flag column to a value of 1. If the value in the neighbourhood column is null and you cannot make a determination based on other information in the dataset, you can set the neighbourhood_flag column to a value of 2 to indicate that the row should be included but the neighbourhood value is null.

You can also use the latitude and longitude columns in the dataset to help match values in the neighbourhood column to valid neighbourhoods in the neighbourhood_list. However, you should be aware that the latitude and longitude values may also contain errors or noise, so you should exercise caution when using these columns to clean up the neighbourhood column.

In [13]:
neighbourhood_list = [ 'Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View', 'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater',    'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park',    'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side',    'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood',    'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale',    'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park',    'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore',    'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa',    'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park',    'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn',    'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights',    'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn',    'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland',    'West Englewood', 'Calumet Heights', 'Auburn Gresham', 'Fuller Park',    'Avalon Park', 'Burnside', 'Clearing', 'Gage Park', 'West Pullman',    'Washington Heights', 'East Side']
print(neighbourhood_list)

['Hyde Park', 'West Town', 'Lincoln Park', 'Near West Side', 'Lake View', 'Dunning', 'Rogers Park', 'Logan Square', 'Uptown', 'Edgewater', 'North Center', 'Albany Park', 'West Ridge', 'Pullman', 'Irving Park', 'Beverly', 'Lower West Side', 'Near South Side', 'Near North Side', 'Grand Boulevard', 'Bridgeport', 'Humboldt Park', 'Chatham', 'Kenwood', 'Loop', 'West Lawn', 'Lincoln Square', 'Woodlawn', 'Avondale', 'Forest Glen', 'Portage Park', 'East Garfield Park', 'Washington Park', 'North Lawndale', 'Armour Square', 'South Lawndale', 'South Shore', 'Morgan Park', 'South Deering', 'West Garfield Park', 'Hermosa', 'Mckinley Park', 'Douglas', 'Hegewisch', 'West Elsdon', 'Norwood Park', 'Garfield Ridge', 'Austin', 'Belmont Cragin', 'Jefferson Park', 'Ashburn', 'Greater Grand Crossing', 'North Park', 'Oakland', 'Archer Heights', 'Edison Park', 'Englewood', 'Ohare', 'Brighton Park', 'Chicago Lawn', 'New City', 'South Chicago', 'Mount Greenwood', 'Montclare', 'Roseland', 'West Englewood', 'Calu

In [14]:
def clean_neighbourhood(df):
    neighbourhood_check = df[df.neighbourhood_flag==0]
    neighbourhood_check = neighbourhood_check[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
    neigh_id_df=neighbourhood_check[["id","neighbourhood"]]
    neigh_ind_ls=list(neigh_id_df.index)
    
    for neigh_ind in neigh_ind_ls:
        sus_neigh_ipt=df.loc[neigh_ind,"neighbourhood"]

        try:
            
            temp_ls=[]
            for neighbour in neighbourhood_list:
                temp_ls.append(distance.sim(sus_neigh_ipt, neighbour))
            if max(temp_ls)>0.5:
                df.loc[neigh_ind,"neighbourhood"]=neighbourhood_list[temp_ls.index(max(temp_ls))]
            else:
                df.loc[neigh_ind,"neighbourhood_flag"]=1
        except:
            df.loc[neigh_ind,"neighbourhood_flag"]=2
        
    return df

In [15]:
airbnb_pd = clean_neighbourhood(airbnb_pd)

In [16]:
airbnb_pd

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,2.89,0,0,1,0,0,0,0,0,0
1,7746,84042,https://www.airbnb.com/rooms/84042,20230319041143,2023-03-19,city scrape,The Explorer Room,This is a peaceful/private bedroom (in a share...,Accessible to everything Chicago has to offer....,https://a0.muscache.com/pictures/13143585/e400...,...,0.85,0,0,1,0,0,0,0,0,0
2,7651,145659,https://www.airbnb.com/rooms/145659,20230319041143,2023-03-19,city scrape,Trendy Roscoe Village 3BR/2BR walk to shops,Awesome 3 bedroom/2 bathroom in one of Chicago...,Trendy Roscoe Village offers a variety of bout...,https://a0.muscache.com/pictures/65ff405d-3f96...,...,0.65,0,2,1,0,0,0,0,0,0
3,7659,264547,https://www.airbnb.com/rooms/264547,20230319041143,2023-03-19,city scrape,Steps From the Best Chicago Offers,Hotel type amenities (in excess) upon arrival....,"It is Chicago's hottest area, centrally locate...",https://a0.muscache.com/pictures/2617984/d552d...,...,0.51,0,0,1,0,0,0,0,0,0
4,7662,290155,https://www.airbnb.com/rooms/290155,20230319041143,2023-03-19,previous scrape,15 Minutes to Everything. Skyline View. Quiet.,Discounts and free nights for 5 nights or more...,"Our area is quiet, and safe. We have great l...",https://a0.muscache.com/pictures/27e377b7-b7a1...,...,0.37,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,7039,782137041139805370,https://www.airbnb.com/rooms/782137041139805370,20230319041143,2023-03-19,city scrape,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,Lovely 2 bed 1 bath apartment located in the h...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.12,0,0,2,0,0,0,0,0,0
1538,7047,782628636832878491,https://www.airbnb.com/rooms/782628636832878491,20230319041143,2023-03-19,city scrape,"Steps to Shop, Eat, Train | Easy Access | Zencity",This studio has “vintage” written all over it—...,"Replete with sushi, wine bars, breweries, and ...",https://a0.muscache.com/pictures/prohost-api/H...,...,3.20,0,0,1,0,0,0,0,0,0
1539,7049,782643895516370805,https://www.airbnb.com/rooms/782643895516370805,20230319041143,2023-03-19,city scrape,Old Town Oasis,This Old Town Oasis is a perfect escape from t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.56,0,0,2,0,0,0,0,0,0
1540,7065,785423932330914663,https://www.airbnb.com/rooms/785423932330914663,20230319041143,2023-03-19,city scrape,"River North 1br w/ gym, pool & roof, nr Riverwalk",Show up and start living from day one in Chica...,This furnished apartment is located in River N...,https://a0.muscache.com/pictures/prohost-api/H...,...,NaN,0,0,1,0,0,0,0,0,0


# Neighbourhood checking

This query should return zero rows once you implement the cleaning process

In [17]:
neighbourhood_check = airbnb_pd[airbnb_pd.neighbourhood_flag==0]
neighbourhood_check = neighbourhood_check.loc[neighbourhood_check.neighbourhood.apply(lambda x:x not in neighbourhood_list)]
neighbourhood_check[["id","neighbourhood"]]

,id,neighbourhood


In [18]:
airbnb_pd.groupby("neighbourhood_flag").count()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
neighbourhood_flag,,,,,,,,,,,,,,,,,,,,,
1,1188,1188,1188,1188,1188,1188,1187,1188,1107,1188,...,1188,995,1188,1188,1188,1188,1188,1188,1188,1188
2,354,354,354,354,354,354,353,354,4,354,...,354,278,354,354,354,354,354,354,354,354


In [19]:
airbnb_pd[airbnb_pd.neighbourhood_flag=="0"]

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag


# cleanup latitude and longitude
The latitude and longitude values in the dataset must fall within the range of -90 to +90 for latitude and -180 to +180 for longitude to ensure that they meet the criteria for analysis. We have provided a check number function to validate the latitude and longitude columns. Any values outside of these ranges should be cleaned to meet the criteria.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting latitude_flag or longitude_flag to 1 or 2, respectively.

In [20]:
def check_number(x,start=-90,end=90):
    try:
        temp_x = float(x)
        #if (start <= temp_x <= end) == True:
        #    return True
        return (start <= temp_x <= end)
    except:
        return False

In [21]:
def clean_latitude(df):
    
    lat_check_pd = df[df.latitude_flag==0]
    lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
    wrong_lat_df=lat_check_pd[["id","latitude"]]
    lat_ind_ls=list(wrong_lat_df.index)

    for lat_id in lat_ind_ls:
        pre_lati=df.loc[lat_id,"latitude"]
        if pre_lati==np.nan:
            df.loc[lat_id,"latitude_flag"]=2
        else:
            try:
                revised=float(''.join(c for c in pre_lati if (c.isdigit() or c =='.')))
                if check_number(revised)==True:
                    df.loc[lat_id,"latitude"]=revised
                else:
                    df.loc[lat_id,"latitude_flag"]=1
        
            except:
                print(lat_id)
    return df

In [22]:
airbnb_pd = clean_latitude(airbnb_pd)

18
19
50
57
73
74
101
107
158
168
195
217
218
228
229
243
244
245
257
316
355
369
376
382
404
420
445
446
472
473
504
509
534
544
550
551
570
578
582
597
603
631
632
672
677
678
687
716
717
718
739
747
779
816
849
850
864
891
897
901
917
936
937
949
952
999
1010
1020
1029
1032
1037
1077
1079
1088
1089
1101
1108
1114
1150
1174
1207
1227
1249
1255
1263
1264
1280
1310
1331
1406
1491
1493
1505
1536
1540


# Latitude checking

This query should return zero rows once you implement the cleaning process

In [23]:
airbnb_pd.groupby("latitude_flag").count()

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
latitude_flag,,,,,,,,,,,,,,,,,,,,,
0,1542,1542,1542,1542,1542,1542,1540,1542,1111,1542,...,1542,1273,1542,1542,1542,1542,1542,1542,1542,1542


In [24]:
lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
lat_check_pd = lat_check_pd[lat_check_pd.latitude.apply(lambda x:check_number(x,-90,90))==False]
lat_check_pd[["id","latitude"]]

,id,latitude
18,2320027,4190798.0
19,2320027,4190798.0
50,6419072,4195533.0
57,6716705,4194072.0
73,7825308,419582.0
...,...,...
1491,760221018437590687,4189038.0
1493,761271176306363398,418815325.0
1505,766069261296774780,4190175.0
1536,782097918539684046,4189052.0


In [25]:
lat_check_pd = airbnb_pd[airbnb_pd.latitude_flag==0]
lat_check_pd

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,2.89,0,0,1,0,0,0,0,0,0
1,7746,84042,https://www.airbnb.com/rooms/84042,20230319041143,2023-03-19,city scrape,The Explorer Room,This is a peaceful/private bedroom (in a share...,Accessible to everything Chicago has to offer....,https://a0.muscache.com/pictures/13143585/e400...,...,0.85,0,0,1,0,0,0,0,0,0
2,7651,145659,https://www.airbnb.com/rooms/145659,20230319041143,2023-03-19,city scrape,Trendy Roscoe Village 3BR/2BR walk to shops,Awesome 3 bedroom/2 bathroom in one of Chicago...,Trendy Roscoe Village offers a variety of bout...,https://a0.muscache.com/pictures/65ff405d-3f96...,...,0.65,0,2,1,0,0,0,0,0,0
3,7659,264547,https://www.airbnb.com/rooms/264547,20230319041143,2023-03-19,city scrape,Steps From the Best Chicago Offers,Hotel type amenities (in excess) upon arrival....,"It is Chicago's hottest area, centrally locate...",https://a0.muscache.com/pictures/2617984/d552d...,...,0.51,0,0,1,0,0,0,0,0,0
4,7662,290155,https://www.airbnb.com/rooms/290155,20230319041143,2023-03-19,previous scrape,15 Minutes to Everything. Skyline View. Quiet.,Discounts and free nights for 5 nights or more...,"Our area is quiet, and safe. We have great l...",https://a0.muscache.com/pictures/27e377b7-b7a1...,...,0.37,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,7039,782137041139805370,https://www.airbnb.com/rooms/782137041139805370,20230319041143,2023-03-19,city scrape,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,Lovely 2 bed 1 bath apartment located in the h...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.12,0,0,2,0,0,0,0,0,0
1538,7047,782628636832878491,https://www.airbnb.com/rooms/782628636832878491,20230319041143,2023-03-19,city scrape,"Steps to Shop, Eat, Train | Easy Access | Zencity",This studio has “vintage” written all over it—...,"Replete with sushi, wine bars, breweries, and ...",https://a0.muscache.com/pictures/prohost-api/H...,...,3.20,0,0,1,0,0,0,0,0,0
1539,7049,782643895516370805,https://www.airbnb.com/rooms/782643895516370805,20230319041143,2023-03-19,city scrape,Old Town Oasis,This Old Town Oasis is a perfect escape from t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.56,0,0,2,0,0,0,0,0,0
1540,7065,785423932330914663,https://www.airbnb.com/rooms/785423932330914663,20230319041143,2023-03-19,city scrape,"River North 1br w/ gym, pool & roof, nr Riverwalk",Show up and start living from day one in Chica...,This furnished apartment is located in River N...,https://a0.muscache.com/pictures/prohost-api/H...,...,NaN,0,0,1,0,0,0,0,0,0


In [26]:
lat_check_pd[lat_check_pd.latitude.apply(lambda x:(-90<float(x)<90)==False)]

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
18,18,2320027,https://www.airbnb.com/rooms/2320027,20230319041143,2023-03-19,previous scrape,Distinctive wicker park attic,This vintage rehab is just steps from the nigh...,"The restaurants, park, and shopping, are some ...",https://a0.muscache.com/pictures/958ceb18-5782...,...,1.50,0,0,1,0,0,0,0,0,0
19,18,2320027,https://www.airbnb.com/rooms/2320027,20230319041143,2023-03-19,previous scrape,Distinctive wicker park attic,This vintage rehab is just steps from the nigh...,"The restaurants, park, and shopping, are some ...",https://a0.muscache.com/pictures/958ceb18-5782...,...,1.50,1,0,1,0,0,0,0,0,0
50,196,6419072,https://www.airbnb.com/rooms/6419072,20230319041143,2023-03-19,previous scrape,Unique and Cozy updated home,"Hi! Welcome to my home. Quiet and homey, upd...",NaN,https://a0.muscache.com/pictures/4c37dd2e-cfe0...,...,0.11,0,0,2,0,0,0,0,0,0
57,228,6716705,https://www.airbnb.com/rooms/6716705,20230319041143,2023-03-19,city scrape,3BD/2BA (vs2) Perfect 4 Families/walk to shops,This is our 3 bedroom/2 bathroom fully furnish...,Trendy Roscoe Village offers boutique shopping...,https://a0.muscache.com/pictures/a4decda4-c81a...,...,0.55,0,2,1,0,0,0,0,0,0
73,300,7825308,https://www.airbnb.com/rooms/7825308,20230319041143,2023-03-19,city scrape,Screened living room with sofa bed /Blue Line /,I live on the first floor of a very quiet buil...,"Nice ,quiet and safe Old Irwing Park neighbor...",https://a0.muscache.com/pictures/100419843/cd4...,...,0.66,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,6817,760221018437590687,https://www.airbnb.com/rooms/760221018437590687,20230319041143,2023-03-19,city scrape,Lovely Huge 1bedroom Apartment Chicago Downtown,Enjoy a stylish experience at this centrally-l...,This unique place has a style all its own. <br...,https://a0.muscache.com/pictures/miso/Hosting-...,...,0.77,0,0,1,0,0,0,0,0,0
1493,6830,761271176306363398,https://www.airbnb.com/rooms/761271176306363398,20230319041143,2023-03-19,city scrape,Entire Condo 3 bed/2 bath in West Loop Chicago,*Condo is located 1-2 blocks from the United C...,*West loop/Near West Side neighborhood<br />*N...,https://a0.muscache.com/pictures/7debf38b-d951...,...,0.53,0,2,1,0,0,0,0,0,0
1505,6882,766069261296774780,https://www.airbnb.com/rooms/766069261296774780,20230319041143,2023-03-19,city scrape,Old-World Unit 3 Downtown!,Our beautiful Brownstone Apartment is located ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,0.81,0,0,2,0,0,0,0,0,0
1536,7038,782097918539684046,https://www.airbnb.com/rooms/782097918539684046,20230319041143,2023-03-19,city scrape,Downtown Luxury Condo,Check out this unique luxury condo minutes awa...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,0.43,0,0,2,0,0,0,0,0,0


In [27]:
lat_check_pd

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,2.89,0,0,1,0,0,0,0,0,0
1,7746,84042,https://www.airbnb.com/rooms/84042,20230319041143,2023-03-19,city scrape,The Explorer Room,This is a peaceful/private bedroom (in a share...,Accessible to everything Chicago has to offer....,https://a0.muscache.com/pictures/13143585/e400...,...,0.85,0,0,1,0,0,0,0,0,0
2,7651,145659,https://www.airbnb.com/rooms/145659,20230319041143,2023-03-19,city scrape,Trendy Roscoe Village 3BR/2BR walk to shops,Awesome 3 bedroom/2 bathroom in one of Chicago...,Trendy Roscoe Village offers a variety of bout...,https://a0.muscache.com/pictures/65ff405d-3f96...,...,0.65,0,2,1,0,0,0,0,0,0
3,7659,264547,https://www.airbnb.com/rooms/264547,20230319041143,2023-03-19,city scrape,Steps From the Best Chicago Offers,Hotel type amenities (in excess) upon arrival....,"It is Chicago's hottest area, centrally locate...",https://a0.muscache.com/pictures/2617984/d552d...,...,0.51,0,0,1,0,0,0,0,0,0
4,7662,290155,https://www.airbnb.com/rooms/290155,20230319041143,2023-03-19,previous scrape,15 Minutes to Everything. Skyline View. Quiet.,Discounts and free nights for 5 nights or more...,"Our area is quiet, and safe. We have great l...",https://a0.muscache.com/pictures/27e377b7-b7a1...,...,0.37,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,7039,782137041139805370,https://www.airbnb.com/rooms/782137041139805370,20230319041143,2023-03-19,city scrape,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,Lovely 2 bed 1 bath apartment located in the h...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.12,0,0,2,0,0,0,0,0,0
1538,7047,782628636832878491,https://www.airbnb.com/rooms/782628636832878491,20230319041143,2023-03-19,city scrape,"Steps to Shop, Eat, Train | Easy Access | Zencity",This studio has “vintage” written all over it—...,"Replete with sushi, wine bars, breweries, and ...",https://a0.muscache.com/pictures/prohost-api/H...,...,3.20,0,0,1,0,0,0,0,0,0
1539,7049,782643895516370805,https://www.airbnb.com/rooms/782643895516370805,20230319041143,2023-03-19,city scrape,Old Town Oasis,This Old Town Oasis is a perfect escape from t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.56,0,0,2,0,0,0,0,0,0
1540,7065,785423932330914663,https://www.airbnb.com/rooms/785423932330914663,20230319041143,2023-03-19,city scrape,"River North 1br w/ gym, pool & roof, nr Riverwalk",Show up and start living from day one in Chica...,This furnished apartment is located in River N...,https://a0.muscache.com/pictures/prohost-api/H...,...,NaN,0,0,1,0,0,0,0,0,0


In [28]:
def clean_longitude(df):
    lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
    lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
    wrong_long_df=lon_check_pd[["id","longitude"]]
    long_ind_ls=list(wrong_long_df.index)

    for long_id in long_ind_ls:
        pre_long=df.loc[long_id,"longitude"]
        if pre_long==np.nan:
            df.loc[long_id,"longitude_flag"]=2
        else:
            try:
                revised=float(''.join(c for c in pre_long if (c.isdigit() or c =='.')))
                if check_number(revised,-180,180)==True:
                    df.loc[long_id,"longitude"]=revised
                else:
                    df.loc[long_id,"longitude_flag"]=1
        
            except:
                print(long_id)
        
    return df

In [29]:
airbnb_pd = clean_longitude(airbnb_pd)

19
22
63
86
117
131
132
162
163
176
187
195
196
205
262
272
311
313
352
358
359
385
386
389
429
492
493
573
595
599
615
620
646
649
685
690
702
709
716
717
719
724
752
757
772
773
780
792
832
858
874
876
904
912
914
938
941
950
951
958
965
987
1002
1003
1026
1039
1051
1053
1073
1111
1117
1161
1170
1172
1198
1199
1262
1290
1321
1346
1347
1353
1399
1404
1421
1453
1457
1462
1471
1507


# Longitude checking

This query should return zero rows once you implement the cleaning process

In [30]:
lon_check_pd = airbnb_pd[airbnb_pd.longitude_flag==0]
lon_check_pd = lon_check_pd[lon_check_pd.longitude.apply(lambda x:check_number(x,-180,180))==False]
lon_check_pd[["id","longitude"]]

,id,longitude
19,2320027,NaN
22,2679456,-8.762121e+06
63,6897078,-8.767139e+07
86,9825135,-8.769546e+06
117,12825187,-8.763936e+06
...,...,...
1453,741628256931082742,-8.769425e+06
1457,742513144401965505,-8.765269e+08
1462,744991422152540096,-8.763672e+06
1471,748671507545699034,-8.765600e+04


In [31]:
-180 <= float(-8.768182e+06) <= 180

False

# cleanup room type
The "room_type" column in the dataset should contain one of the values defined in the list of allowed_room_type provided by the authority: ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']. Any value outside of this list needs to be adjusted to one of the allowed values.

If you are unsure about how to adjust the value or cannot find a suitable value, you can flag the row for deletion by setting the value of room_type_flag to 1. If the "room_type" column has a null value and you cannot decide on an appropriate value, you can set the value of room_type_flag to 2.

In [32]:
allowed_room_type = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']

In [33]:
def clean_room_type(df):
    room_type_pd = df[df.room_type_flag==0]
    room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
    room_type_pd[["id","room_type"]]
    room_id_ls=list(room_type_pd[["id","room_type"]].index)
    
    for room_ind in room_id_ls:
        sus_room_ipt=df.loc[room_ind,"room_type"]

        try:
            temp_ls=[]
            for room in allowed_room_type:
                temp_ls.append(distance.sim(sus_room_ipt, room))
            if max(temp_ls)>0.5:
                df.loc[room_ind,"room_type"]=allowed_room_type[temp_ls.index(max(temp_ls))]
            else:
                df.loc[room_ind,"room_type_flag"]=1
        except:
            df.loc[room_ind,"room_type_flag"]=2
        
    return df

In [34]:
airbnb_pd = clean_room_type(airbnb_pd)

# room_type checking

This query should return zero rows once you implement the cleaning process

In [35]:
room_type_pd = airbnb_pd[airbnb_pd.room_type_flag==0]
room_type_pd = room_type_pd[room_type_pd.room_type.apply(lambda x: x not in allowed_room_type)]
room_type_pd[["id","room_type"]]

,id,room_type


In [36]:
#airbnb_pd[airbnb_pd.room_type_flag==0].room_type.unique()

# cleanup minimum_nights and number_of_reviews

The columns "minimum_nights" and "number_of_reviews" should both be integer values. "minimum_nights" should be a value between 1 and the number of days in a year (365), while "number_of_reviews" should be a value between 0 and 999999.

To check if these columns meet the criteria, we have provided a "check_integer" function. Any values that do not meet the criteria should be cleaned to meet the criteria for analysis.

If you are unsure what to do with a value or if it is a null value, you can flag the row for deletion by setting "minimum_nights_flag" or "number_of_reviews_flag" to 1 or 2, respectively.

In [37]:
def check_integer(x,start=1,end=365):
    try:
        temp_x = int(x)
        if start <= temp_x <= end:
            return True
    except:
        return False

In [38]:
def clean_minimum_nights(df):
    min_check_pd = df[df.minimum_nights_flag==0]
    min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,1,365))==False]
    wrong_min_ng= min_check_pd[["id","minimum_nights"]]
    ng_ind_ls=list(wrong_min_ng.index)

    for ng_id in ng_ind_ls:
        pre_ng=df.loc[ng_id,"minimum_nights"]
        
        try:
            if pre_ng==np.nan or math.isnan(pre_ng):
                # print("here:" +str(ng_id))
                df.loc[ng_id,"minimum_nights_flag"]=2
        except:
            try:
                revised=int(''.join(c for c in pre_ng if c.isdigit()))
                if check_integer(revised)==True:
                    df.loc[ng_id,"minimum_nights"]=revised
                else:
                    df.loc[ng_id,"minimum_nights_flag"]=1
        
            except:
                print(ng_id)
        
    return df

In [39]:
airbnb_pd = clean_minimum_nights(airbnb_pd)

# Minimum nights checking

This query should return zero rows once you implement the cleaning process

In [40]:
min_check_pd = airbnb_pd[airbnb_pd.minimum_nights_flag==0]
min_check_pd = min_check_pd[min_check_pd.minimum_nights.apply(lambda x:check_integer(x,1,365))==False]
min_check_pd[["id","minimum_nights"]]

,id,minimum_nights


In [41]:
def clean_number_of_reviews(df):
    min_check_pd = df[df.number_of_reviews_flag==0]
    min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,1,999999))==False]
    wrong_min_nr=  min_check_pd[["id","number_of_reviews"]]
    nr_ind_ls=list(wrong_min_nr.index)

    for nr_id in nr_ind_ls:
        pre_nr=df.loc[nr_id,"number_of_reviews"]
        try:
            if pre_nr==np.nan or math.isnan(pre_nr):
                df.loc[nr_id,"number_of_reviews_flag"]=2
        except:
            try:
                revised=int(''.join(c for c in pre_nr if c.isdigit()))
                if check_integer(revised)==True:
                    df.loc[nr_id,"number_of_reviews"]=revised
                else:
                    df.loc[nr_id,"number_of_reviews_flag"]=1
        
            except:
                print(nr_id)
    return df

In [42]:
airbnb_pd = clean_number_of_reviews(airbnb_pd)

# Clean number of reviews checking

This query should return zero rows once you implement the cleaning process

In [43]:
min_check_pd = airbnb_pd[airbnb_pd.number_of_reviews_flag==0]
min_check_pd = min_check_pd[min_check_pd.number_of_reviews.apply(lambda x:check_integer(x,1,999999))==False]
min_check_pd[["id","number_of_reviews"]]

,id,number_of_reviews


# cleanup last_review

The "last_review" column should be in the format of ISO-date (yyyy-mm-dd). We have provided a "check_date" function to verify the date format.

If a value is outside the date format or is null and you are unsure how to handle it, you can flag the row for deletion by setting the "last_review_flag" to 1 or 2.


In [44]:
from datetime import datetime
def check_date(x,fmt="%Y-%m-%d"):
    try:
        datetime.strptime(x,fmt)
        return True
    except:
        return False

In [45]:
def clean_last_reviews(df):
    last_review_check_pd = df[df.last_review_flag==0]
    last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
    wrong_lr=last_review_check_pd[["id","last_review"]]
    lr_ind_ls=list(wrong_lr.index)
    
    for lr_id in lr_ind_ls:
        pre_lr=df.loc[lr_id,"last_review"]
        try:
            if pre_lr==np.nan or math.isnan(pre_lr):
                df.loc[lr_id,"last_review_flag"]=2
        except:
            pre_lr=pre_lr.replace("/", " ")
            mon_ele=pre_lr.split()[0]
            date_ele=pre_lr.split()[1]
            year_ele=pre_lr.split()[2]
            revised_date="20"+year_ele+"-"+mon_ele+"-"+date_ele
            if check_date(revised_date)==True:
                df.loc[lr_id,"last_review"]=revised_date
            else:
                df.loc[lr_id,"last_review_flag"]=1
    return df

In [46]:
airbnb_pd = clean_last_reviews(airbnb_pd)

# Last Review checking

This query should return zero rows once you implement the cleaning process

In [47]:
last_review_check_pd = airbnb_pd[airbnb_pd.last_review_flag==0]
last_review_check_pd = last_review_check_pd[last_review_check_pd.last_review.apply(lambda x:check_date(x))==False]
last_review_check_pd[["id","last_review"]]

,id,last_review


# save the dataset to csv

In [48]:
airbnb_pd.to_csv("chicago_vert_dataset_cleaned_2.csv")

# columns that potentially will be used for analysis:
id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price

In [49]:
columns_used = ["id","name","host_id","host_name",
                         "neighbourhood","latitude","longitude",
                         "room_type","minimum_nights","number_of_reviews","last_review","price"]

In [50]:
airbnb_pd[columns_used]

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,price
0,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,"Chicago, Illinois, United States",4.190166e+01,-87.680210,Entire home/apt,2,483,2023-02-26,$85.00
1,84042,The Explorer Room,334241,Michael And Veronica,"Chicago, Illinois, United States",4.189615e+01,-87.679340,Private room,3,124,2022-12-11,$89.00
2,145659,Trendy Roscoe Village 3BR/2BR walk to shops,683529,Joe,"Chicago, Illinois, United States",4.194342e+01,-87.681210,Entire home/apt,2,59,2022-12-31,$198.00
3,264547,Steps From the Best Chicago Offers,1387948,Dominic,"Chicago, Illinois, United States",4.188413e+01,-87.643150,Entire home/apt,32,67,2021-10-17,$204.00
4,290155,15 Minutes to Everything. Skyline View. Quiet.,1505878,Bob,"Chicago, Illinois, United States",4.182712e+01,-87.640460,Entire home/apt,32,50,2020-03-17,$130.00
...,...,...,...,...,...,...,...,...,...,...,...,...
1537,782137041139805370,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,215444392,Jenna,NaN,4.187047e+01,-87.662579,Apartment,2,6,2023-03-05,$130.00
1538,782628636832878491,"Steps to Shop, Eat, Train | Easy Access | Zencity",47172572,Zencity,"Chicago, Illinois, United States",4.189682e+01,-87.661130,Entire home/apt,1,8,2023-03-04,$81.00
1539,782643895516370805,Old Town Oasis,169297663,William,NaN,4.190146e+01,-87.636660,Entire home/apt,3,7,2023-03-05,$336.00
1540,785423932330914663,"River North 1br w/ gym, pool & roof, nr Riverwalk",107434423,Blueground,"Chicago, Illinois, United States",4.189052e+07,-87.635955,Entire home/apt,32,0,NaN,$158.00


# Checking and Comparing result to the Groundtruth

In [51]:
check_pd = pd.read_csv("airbnb_sample2_a.csv")

In [52]:
airbnb_pd[airbnb_pd.id_flag==0]

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,2.89,0,0,1,0,0,0,0,0,0
1,7746,84042,https://www.airbnb.com/rooms/84042,20230319041143,2023-03-19,city scrape,The Explorer Room,This is a peaceful/private bedroom (in a share...,Accessible to everything Chicago has to offer....,https://a0.muscache.com/pictures/13143585/e400...,...,0.85,0,0,1,0,0,0,0,0,0
2,7651,145659,https://www.airbnb.com/rooms/145659,20230319041143,2023-03-19,city scrape,Trendy Roscoe Village 3BR/2BR walk to shops,Awesome 3 bedroom/2 bathroom in one of Chicago...,Trendy Roscoe Village offers a variety of bout...,https://a0.muscache.com/pictures/65ff405d-3f96...,...,0.65,0,2,1,0,0,0,0,0,0
3,7659,264547,https://www.airbnb.com/rooms/264547,20230319041143,2023-03-19,city scrape,Steps From the Best Chicago Offers,Hotel type amenities (in excess) upon arrival....,"It is Chicago's hottest area, centrally locate...",https://a0.muscache.com/pictures/2617984/d552d...,...,0.51,0,0,1,0,0,0,0,0,0
4,7662,290155,https://www.airbnb.com/rooms/290155,20230319041143,2023-03-19,previous scrape,15 Minutes to Everything. Skyline View. Quiet.,Discounts and free nights for 5 nights or more...,"Our area is quiet, and safe. We have great l...",https://a0.muscache.com/pictures/27e377b7-b7a1...,...,0.37,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,7039,782137041139805370,https://www.airbnb.com/rooms/782137041139805370,20230319041143,2023-03-19,city scrape,UIC / Little Italy - Adorable 2 Bed 1 Bath Apt.,Lovely 2 bed 1 bath apartment located in the h...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.12,0,0,2,0,0,0,0,0,1
1538,7047,782628636832878491,https://www.airbnb.com/rooms/782628636832878491,20230319041143,2023-03-19,city scrape,"Steps to Shop, Eat, Train | Easy Access | Zencity",This studio has “vintage” written all over it—...,"Replete with sushi, wine bars, breweries, and ...",https://a0.muscache.com/pictures/prohost-api/H...,...,3.20,0,0,1,0,0,0,0,0,0
1539,7049,782643895516370805,https://www.airbnb.com/rooms/782643895516370805,20230319041143,2023-03-19,city scrape,Old Town Oasis,This Old Town Oasis is a perfect escape from t...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,...,2.56,0,0,2,0,0,0,0,0,0
1540,7065,785423932330914663,https://www.airbnb.com/rooms/785423932330914663,20230319041143,2023-03-19,city scrape,"River North 1br w/ gym, pool & roof, nr Riverwalk",Show up and start living from day one in Chica...,This furnished apartment is located in River N...,https://a0.muscache.com/pictures/prohost-api/H...,...,NaN,0,0,1,0,0,0,0,2,0


In [53]:
check_pd

,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,7,1921670,https://www.airbnb.com/rooms/1921670,20230319041143,2023-03-19,city scrape,The Avondale Garden Unit,3 Miles from Wrigley!!! Check out this private...,The area is very popular with restaurants and ...,https://a0.muscache.com/pictures/06afc5da-cc29...,...,4.96,4.81,4.88,R18000036958,t,8,8,0,0,2.91
1,11,2062638,https://www.airbnb.com/rooms/2062638,20230319041143,2023-03-19,city scrape,Downtown Urban Oasis 3BR + Huge Deck & Parking!,**PLEASE READ DESCRIPTION AND CLICK 'CONTACT H...,"River West is one of the trendiest, most centr...",https://a0.muscache.com/pictures/4233f06b-0eda...,...,4.93,4.80,4.67,2510880,f,2,2,0,0,1.74
2,13,2139407,https://www.airbnb.com/rooms/2139407,20230319041143,2023-03-19,city scrape,Urban Farm Fun! Walk to Metra/CTA,This cozy one bedroom apartment has a full kit...,East Garfield Park is definitely the inner cit...,https://a0.muscache.com/pictures/miso/Hosting-...,...,4.73,4.08,4.56,R22000078847,t,6,6,0,0,4.72
3,15,2298310,https://www.airbnb.com/rooms/2298310,20230319041143,2023-03-19,city scrape,"Hyde Park / Kenwood, U of Chicago. Free bike.",A second-floor bedroom in a 3- level townhouse...,NaN,https://a0.muscache.com/pictures/d5e29ef0-4f3c...,...,4.94,4.57,4.72,R21000072908,f,15,3,12,0,0.43
4,18,2320027,https://www.airbnb.com/rooms/2320027,20230319041143,2023-03-19,previous scrape,Distinctive wicker park attic,This vintage rehab is just steps from the nigh...,"The restaurants, park, and shopping, are some ...",https://a0.muscache.com/pictures/958ceb18-5782...,...,4.87,4.82,4.57,R12345678901,f,2,1,1,0,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,7710,1195259,https://www.airbnb.com/rooms/1195259,20230319041143,2023-03-19,city scrape,Wicker Park - Garden At The Heart of It All,Boutique experience in the heart of the action...,Absolute heart of Wicker Park!,https://a0.muscache.com/pictures/67372160/1587...,...,4.91,4.95,4.85,R19000037818,f,2,2,0,0,1.72
1465,7712,1304049,https://www.airbnb.com/rooms/1304049,20230319041143,2023-03-19,city scrape,Private room near UIC Medical Dist.,Our posh & comfortable apartment has a true ci...,This neighborhood has permit parking. We prov...,https://a0.muscache.com/pictures/21979262/e9d5...,...,4.71,4.58,4.62,R17000015307,f,1,0,1,0,1.02
1466,7718,1461451,https://www.airbnb.com/rooms/1461451,20230319041143,2023-03-19,city scrape,02b. Twin bed in Coed Space,This is for a single twin bed in a SHARED bedr...,We are in the far north neighborhood of Chicag...,https://a0.muscache.com/pictures/da1a79e0-ac55...,...,4.79,4.49,4.63,R21000075752,f,14,0,5,9,1.53
1467,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,4.87,4.88,4.75,R21000075737,f,1,1,0,0,2.89


In [54]:
list(filter(lambda x:x.endswith("flag"),airbnb_pd.columns))

['id_flag',
 'host_id_flag',
 'neighbourhood_flag',
 'latitude_flag',
 'longitude_flag',
 'minimum_nights_flag',
 'number_of_reviews_flag',
 'last_review_flag',
 'room_type_flag']

In [55]:
airbnb_output_list = airbnb_pd[(airbnb_pd.id_flag==0)&(airbnb_pd.host_id_flag==0)&
          (airbnb_pd.neighbourhood_flag==0)&(airbnb_pd.latitude_flag==0)&
         (airbnb_pd.longitude_flag==0)&(airbnb_pd.minimum_nights_flag==0)&
         (airbnb_pd.number_of_reviews_flag==0)&(airbnb_pd.last_review_flag==0)&
         (airbnb_pd.room_type_flag==0)
         ]

In [56]:
merge_check = check_pd.merge(airbnb_output_list,left_on="id",right_on="id")

In [57]:
set_col = set(check_pd.columns)
set_col = set_col  - {"id"}

In [58]:
#merge_check[f"{x}_x"].fillna("").apply(lambda x:str(x))

In [59]:
for x in set_col:
    try:
        #print(x)
        compare = merge_check[f"{x}_x"].fillna("").apply(lambda x: str(x)) != merge_check[f"{x}_y"].fillna("").apply(lambda x:str(x))
        print(x,sum(compare))
    except:
        print("failed",x)
        pass

host_is_superhost 0
review_scores_communication 0
Unnamed: 0 0
calculated_host_listings_count_private_rooms 0
accommodates 0
review_scores_location 0
host_picture_url 0
picture_url 0
bedrooms 0
host_listings_count 0
review_scores_cleanliness 0
calculated_host_listings_count_entire_homes 0
bathrooms 0
maximum_nights 0
latitude 0
property_type 0
name 0
host_total_listings_count 0
maximum_maximum_nights 0
minimum_maximum_nights 0
description 0
host_url 0
license 0
number_of_reviews_ltm 0
instant_bookable 0
host_about 0
host_thumbnail_url 0
host_has_profile_pic 0
neighbourhood_group_cleansed 0
host_location 0
neighbourhood_cleansed 0
amenities 0
availability_30 0
host_neighbourhood 0
availability_365 0
host_response_rate 0
longitude 0
review_scores_rating 0
host_verifications 0
price 0
review_scores_checkin 0
neighborhood_overview 0
host_acceptance_rate 0
source 0
room_type 0
review_scores_value 0
minimum_minimum_nights 0
number_of_reviews 0
maximum_nights_avg_ntm 0
bathrooms_text 0
host_r

In [60]:
missing_check = check_pd.merge(airbnb_output_list,left_on="id",right_on="id",how="left")
#missing_check[missing_check.]
missing_check[missing_check.license_y.isnull()]

,Unnamed: 0_x,id,listing_url_x,scrape_id_x,last_scraped_x,source_x,name_x,description_x,neighborhood_overview_x,picture_url_x,...,reviews_per_month_y,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag,room_type_flag
0,7,1921670,https://www.airbnb.com/rooms/1921670,20230319041143,2023-03-19,city scrape,The Avondale Garden Unit,3 Miles from Wrigley!!! Check out this private...,The area is very popular with restaurants and ...,https://a0.muscache.com/pictures/06afc5da-cc29...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,2062638,https://www.airbnb.com/rooms/2062638,20230319041143,2023-03-19,city scrape,Downtown Urban Oasis 3BR + Huge Deck & Parking!,**PLEASE READ DESCRIPTION AND CLICK 'CONTACT H...,"River West is one of the trendiest, most centr...",https://a0.muscache.com/pictures/4233f06b-0eda...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,2139407,https://www.airbnb.com/rooms/2139407,20230319041143,2023-03-19,city scrape,Urban Farm Fun! Walk to Metra/CTA,This cozy one bedroom apartment has a full kit...,East Garfield Park is definitely the inner cit...,https://a0.muscache.com/pictures/miso/Hosting-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,2298310,https://www.airbnb.com/rooms/2298310,20230319041143,2023-03-19,city scrape,"Hyde Park / Kenwood, U of Chicago. Free bike.",A second-floor bedroom in a 3- level townhouse...,NaN,https://a0.muscache.com/pictures/d5e29ef0-4f3c...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,2320027,https://www.airbnb.com/rooms/2320027,20230319041143,2023-03-19,previous scrape,Distinctive wicker park attic,This vintage rehab is just steps from the nigh...,"The restaurants, park, and shopping, are some ...",https://a0.muscache.com/pictures/958ceb18-5782...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,7710,1195259,https://www.airbnb.com/rooms/1195259,20230319041143,2023-03-19,city scrape,Wicker Park - Garden At The Heart of It All,Boutique experience in the heart of the action...,Absolute heart of Wicker Park!,https://a0.muscache.com/pictures/67372160/1587...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1465,7712,1304049,https://www.airbnb.com/rooms/1304049,20230319041143,2023-03-19,city scrape,Private room near UIC Medical Dist.,Our posh & comfortable apartment has a true ci...,This neighborhood has permit parking. We prov...,https://a0.muscache.com/pictures/21979262/e9d5...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1466,7718,1461451,https://www.airbnb.com/rooms/1461451,20230319041143,2023-03-19,city scrape,02b. Twin bed in Coed Space,This is for a single twin bed in a SHARED bedr...,We are in the far north neighborhood of Chicag...,https://a0.muscache.com/pictures/da1a79e0-ac55...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1467,7738,7126,https://www.airbnb.com/rooms/7126,20230319041143,2023-03-19,city scrape,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
merge_check[["price_x","price_y"]]

,price_x,price_y


In [62]:
merge_check.price_x

Series([], Name: price_x, dtype: object)

In [63]:
merge_check[["room_type_x","room_type_y"]]

,room_type_x,room_type_y


In [64]:
airbnb_output_list.groupby(["room_type","room_type_flag"]).count()

,,Unnamed: 0,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,calculated_host_listings_count_shared_rooms,reviews_per_month,id_flag,host_id_flag,neighbourhood_flag,latitude_flag,longitude_flag,minimum_nights_flag,number_of_reviews_flag,last_review_flag
room_type,room_type_flag,,,,,,,,,,,,,,,,,,,,,
